In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep

In [2]:
url = "https://newyork.craigslist.org/search/sss?query=bikes"
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [3]:
results_page = BeautifulSoup(response.content,'lxml')

# find the total number of posts
results_num = results_page.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)

prices = []
neighborhood = []
urls = []
dates = []

pages = np.arange(0, results_total+1, 120)

# Loop through all pages
for page in pages:
    
    response = requests.get(url 
                   + "s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(0.5)

    if response.status_code != 200:
        print(('Request: {}; Status code: {}'.format(requests, response.status_code)))
    
    page_html = BeautifulSoup(response.text, 'html.parser')

    posts = results_page.find_all('li', class_= 'result-row')

    for post in posts:
        prices.append(int(post.find('span', class_="result-price").text[1:].replace(',', '')))
        neighborhood.append(post.find('span', class_="result-hood").text)
        urls.append(post.find('a', class_="result-title hdrlnk")['href'])
        dates.append(post.find('time', class_="result-date")['datetime'])

In [4]:
df = pd.DataFrame({'Price':prices, 'Neighborhood':neighborhood, 'URL':urls, 'Date':dates})
df

,Price,Neighborhood,URL,Date
0,100,(Queens),https://newyork.craigslist.org/que/spo/d/rocka...,2022-04-13 18:24
1,75,(Brooklyn Queens Manhattan Bronx),https://newyork.craigslist.org/brk/mpo/d/astor...,2022-04-13 15:52
2,60,(Brooklyn),https://newyork.craigslist.org/brk/bik/d/brook...,2022-04-13 13:19
3,0,(Shelton),https://newyork.craigslist.org/fct/wan/d/bmx-b...,2022-04-13 13:05
4,200,(Ridgewood),https://newyork.craigslist.org/brk/bik/d/ridge...,2022-04-13 11:54
...,...,...,...,...
3115,190,(BROOKLYN),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-31 11:35
3116,0,(westchester),https://newyork.craigslist.org/wch/wan/d/maybr...,2022-03-31 09:14
3117,0,(westchester),https://newyork.craigslist.org/wch/wan/d/maybr...,2022-03-31 09:14
3118,250,(West Village),https://newyork.craigslist.org/mnh/bik/d/new-y...,2022-03-31 08:58


In [5]:
df = df.where(df['Price']!=0).dropna().reset_index()

In [8]:
for ind in df[338:].index:

    url_ = df.loc[ind, 'URL']
    
    sleep(1)

    response = requests.get(url_, headers={'User-Agent': 'Mozilla/5.0'})

    if response.status_code != 200:
        print(('Request: {}; Status code: {}'.format(requests, response.status_code)))
        print(ind)
        df_old = pd.read_csv('data_craigslist.csv')
        new_df = pd.concat(df_old, df[338:])
        new_df.to_csv('data_craigslist2.csv')
        break

    page_html = BeautifulSoup(response.text, 'html.parser')
    
    all_attributes = page_html.find('p', class_= 'attrgroup')

    # Finds all the attributes
    try:
        attr = all_attributes.find_all('span')

        for att in attr:
            str_attr = att.text.split(':')
            attribute, value = str_attr
            if attribute not in df.columns:
                df[attribute] = [np.nan for _ in range(len(df))]
            df.loc[ind, attribute] = value
    except:
        pass

    # Finds the number of images
    try:
        user_body = page_html.find('section', class_="userbody")
        multi_img = user_body.find('figure', class_="iw multiimage")
        num_img = multi_img.find('span', class_="slider-info").text.split(' ')[-1]
        if 'num image' not in df.columns:
            df['num image'] = [0 for _ in range(len(df))]
        df.loc[ind, 'num image'] = num_img
    except:
        if 'num image' not in df.columns:
            df['num image'] = [0 for _ in range(len(df))]
        df.loc[ind, 'num image'] = 0

    # Find the text
    try:
        user_text = str(page_html.find('section', {'id':"postingbody"}).text).strip().replace('QR Code Link to This Post', '').replace('\n', ' ').strip()
        if 'text' not in df.columns:
                df['text'] = [np.nan for _ in range(len(df))]
        df.loc[ind, 'text'] = user_text
    except:
        pass


Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403
338


PermissionError: [Errno 13] Permission denied: 'data_craigslist.csv'

In [ ]:
df.to_csv('data_craigslist.csv')